In [1]:
import numpy as np
import pandas as pd
from openpyxl.utils import get_column_letter

In [2]:
t = 9 # mm thickness of shell
b = 400 #mm width of shell a=600
E = 66052.82 # MPa Young's modulus (B-Basis value)
nu = 0.34 # Poisson's ratio
alpha = 1.5 # a/b=600/400=1.5 ratio
m = 2 #no. of half sinosoidal waves x-direction
n = 1 #no. of half sinosoidal waves y-direction


In [ ]:
# ───────────────────────────────────────────────────────────────
# 2) Excel → DataFrame 
# ───────────────────────────────────────────────────────────────
file_in = "modified_CBEAM_axial_stress_and_2Delements.xlsx"
sheets_to_update = ["6_redesign_2D"]

In [4]:
# ───────────────────────────────────────────────────────────────
# 3) Vectorised column-wise formula
# ───────────────────────────────────────────────────────────────
with pd.ExcelWriter(file_in, engine="openpyxl", if_sheet_exists="overlay", mode="a") as writer:
    # 1) read sheets
    for sheet in sheets_to_update:
        df = pd.read_excel(file_in,sheet_name=sheet,engine="openpyxl")
        
        #remember first empty columns before adding new one
        first_blank = len(df.columns)

        #force numeric for safety
        for col in ("sig_xx_avg","sig_yy_avg","sig_xy_avg","vonMises"):
            df[col] = pd.to_numeric(df[col], errors='coerce')

        #RF_strengthfactor calculation
        df["RF_strengthfactor"] = 530/(1.5*df["vonMises"]) #1.5 multiplied for safety factor

        #RF_shear calculation
        df["sig_effective"] = (E*(np.pi**2)*(t**2))/(12*(1-(nu**2))*(b**2))
        df["k_shear"] = 5.34 + (4/(alpha**2)) 
        df["sig_crit_shear"] = df["sig_effective"]*df["k_shear"]
        df["RF_shear"] = df["sig_crit_shear"]/(1.5*df["sig_xy_avg"]) #1.5 multiplied for safety factor
        
        #RF_biaxial calculation
        df["beta"] = df["sig_yy_avg"]/df["sig_xx_avg"]

        df["sig_xx_abs"] = df["sig_xx_avg"].abs()

        # boolean masks
        pos = df["beta"] >  0           # β ≥ 0  → m = 1
        neg = df["beta"] <  0           # β <  0 → m = 2
        
        df.loc[pos, "k_biaxial"] = ((1**2+(n**2)*(alpha**2))**2)/((alpha**2)*(1**2+df["beta"]*n**2*alpha**2))
        df.loc[neg, "k_biaxial"] = ((2**2+(n**2)*(alpha**2))**2)/((alpha**2)*(2**2+df["beta"]*n**2*alpha**2))    
            
        #RF_biaxial calculation    
        df["sig_crit_biaxial"] = df["sig_effective"] * df["k_biaxial"]
        df["RF_biaxial"] = df["sig_crit_biaxial"]/(1.5*(df["sig_xx_abs"]))
           
        
        #RF_combined calculation
        df["RF_combined_inverse"]= (1/df["RF_biaxial"]) + (1/df["RF_shear"])**2
        df["RF_combined"] = 1/df["RF_combined_inverse"]
        
        #write new columns side by side to the existing shee
        df[["RF_strengthfactor","sig_effective", "k_shear", "sig_crit_shear", "RF_shear", "beta", "k_biaxial", "sig_crit_biaxial", "RF_biaxial", "RF_combined"]].to_excel(
            writer, sheet_name=sheet, index=False, startcol=first_blank     
        )

         # 6) friendly log
        start_letter = get_column_letter(first_blank + 1)
        end_letter   = get_column_letter(first_blank + 3)
        print(f"{sheet}: wrote sig_crit → RF_combined into {start_letter}:{end_letter}")

print("✓ All listed sheets updated in-place (overlay).")

  

5_redesign_2D: wrote sig_crit → RF_combined into N:P
✓ All listed sheets updated in-place (overlay).
